In [1]:
# This is only valid when the package is not installed
import sys
sys.path.append('../../') # two folders up
import DeepINN as dp
import torch

Using default backend: PyTorch
Using Pytorch:  2.0.1+cu117


### Geometry

In [2]:
# A simple 1D geometry
X = dp.spaces.R1('x')
Line = dp.domains.Interval(X, 0, 1)

In [3]:
left_bc = dp.constraint.DirichletBC(geom = Line,
                                      function = lambda X: torch.tensor([0.0]),
                                      sampling_strategy = "grid",
                                      no_points = 1, # you can use more points. there are conditions to deal with stupid conditions. 
                                      filter_fn = lambda x: x[:] == 0.0)
right_bc = dp.constraint.DirichletBC(geom = Line,
                                      function = lambda X: torch.tensor([1.0]),
                                      sampling_strategy = "grid",
                                      no_points = 1, # you can use more points. there are conditions to deal with stupid conditions. 
                                      filter_fn = lambda x: x[:] == 1.0)
interior_points = dp.constraint.PDE(geom = Line,
                                    sampling_strategy= "grid",
                                    no_points = 20)

In [4]:
# dp.utils.scatter(X, interior_points.sampler_object(), dpi = 50)
# dp.utils.scatter(X, left_bc.sampler_object(), dpi = 50)
# dp.utils.scatter(X, right_bc.sampler_object(), dpi = 50)

### PDE

In [5]:
def laplace(X,y):
    """
    1D Laplace equation.
    u__x = 0 
    """
    dy_x = dp.constraint.Jacobian(X, y)
    dy_xx = dp.constraint.Jacobian(X, y)(i = 0, j = 0)

    return dy_xx

In [6]:
domain = dp.domain.Generic(laplace,
                            interior_points,
                            [left_bc, right_bc])

### Network

In [7]:
activation = "tanh"
initialiser = "Xavier normal"
layer_size = [1] + [5] * 1 + [1]

In [8]:
net = dp.nn.FullyConnected(layer_size, activation, initialiser)
model = dp.Model(domain, net)

In [9]:
optimiser = "adam"
lr=0.001
metrics="MSE"

model.compile(optimiser, lr, metrics, device = "cuda")

/home/hell/Desktop/repos/DeepINN/Tutorials/5. FCNN/../../DeepINN/geometry/samplers/grid_samplers.py:78: UserWarning: First iteration did not find any valid grid points, for
                             the given filter.
                             Will try again with n = 10 * self.n_points. Or
                             else use only random points!
  warnings.warn("""First iteration did not find any valid grid points, for
Domain compiled
Network compiled


In [10]:
model.optimiser_function, model.lr, model.metric

(torch.optim.adam.Adam, 0.001, MSELoss())

In [15]:
model.train(iterations = 100)

Training finished


In [22]:
model.iter = 1
model.train(iterations = 2000)

Iteration: 21 	 BC Loss: 0.2390	 PDE Loss: 0.0063 	 Loss: 0.2453
Iteration: 41 	 BC Loss: 0.2390	 PDE Loss: 0.0063 	 Loss: 0.2453
Iteration: 61 	 BC Loss: 0.2390	 PDE Loss: 0.0062 	 Loss: 0.2452
Iteration: 81 	 BC Loss: 0.2390	 PDE Loss: 0.0061 	 Loss: 0.2451
Iteration: 101 	 BC Loss: 0.2390	 PDE Loss: 0.0061 	 Loss: 0.2451
Iteration: 121 	 BC Loss: 0.2390	 PDE Loss: 0.0060 	 Loss: 0.2450
Iteration: 141 	 BC Loss: 0.2390	 PDE Loss: 0.0060 	 Loss: 0.2449
Iteration: 161 	 BC Loss: 0.2389	 PDE Loss: 0.0059 	 Loss: 0.2449
Iteration: 181 	 BC Loss: 0.2389	 PDE Loss: 0.0059 	 Loss: 0.2448
Iteration: 201 	 BC Loss: 0.2389	 PDE Loss: 0.0059 	 Loss: 0.2448
Training finished
